In [1]:
import tensorflow as tf # type: ignore #
from tensorflow.keras.models import load_model #type: ignore #
import numpy as np
import cv2

modelPath = "C:/Users/aahfa\Documents/personalCode/python/ML_AI/models/humanIdentifyV1.h5"
model = load_model(modelPath)

# Initialize the camera feed (0 is the default camera)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read a frame from the camera
    if not ret:
        break

    # Preprocess the frame for the model
    img = cv2.resize(frame, (224, 224))  # Resize frame to match model's expected input size
    img_array = np.expand_dims(img, axis=0)
    img_array = img_array / 255.0  # Normalize the input

    # Make predictions
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # Annotate the frame with prediction results
    if predicted_class == 0:  # Assuming 0 indicates healthy
        label = 'Human'
        color = (0, 255, 0)  # Green color for healthy
    else:
        label = 'other'
        color = (255, 0, 0)

    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Potato Blight Detection', frame)

    # Break the loop if 'e' is pressed
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 81ms/step
